Silicon Forest Math Series<br/>[Oregon Curriculum Network](http://4dsolutions.net/ocn/)

## Generators and Coroutines

The generator function below shows what most Pythonistas consider the base case for the keyword yield: using it much like return, to provide an object to the generator function's caller, in this case a next prime number, going in sequence.

The trial-by-division algorithm requires keeping a growing sequence of successive primes, and using them to test new candidates.  After taking care of the only even prime, 2, it makes sense to jump forward through the odds, screening out all the composites.

In [1]:
import pprint

def primes():
    """generate successive prime numbers (trial by division)"""
    candidate = 1
    _primes_so_far = [2]     # first prime, only even prime
    yield _primes_so_far[0]  # share it!
    while True:
        candidate += 2    # check odds only from now on
        for prev in _primes_so_far:
            if prev**2 > candidate:
                yield candidate  # new prime!
                _primes_so_far.append(candidate)
                break
            if not divmod(candidate, prev)[1]: # no remainder!
                break                          # done looping
                
p = primes()  # generator function based iterator
pp = pprint.PrettyPrinter(width=40, compact=True)
pp.pprint([next(p) for _ in range(30)])  # next 30 primes please!

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31,
 37, 41, 43, 47, 53, 59, 61, 67, 71, 73,
 79, 83, 89, 97, 101, 103, 107, 109,
 113]


In the code below, we see *(yield)* turned around and used not to return objects, but to receive them.  The parentheses are by convention and suggest a "mouth" bringing something in from outside.

A generator function's .send() method resumes its execution inside its body with an intake of some passed-in object from the caller, the argument to *send*.  In the above example, two callers are nested, the inner one writing a prime to a file, the outer one feeding it next primes for said catalog.

The coroutine decorator may seem a bit mysterious at first.  A generator function does not run any of its code upon being instanced.  No yield statement has yet been encountered, so use of .send(obj) would raise an exception were obj any object but None.  

The decorator has already fed a .send(None) to the generator in question, equivalent to feeding it to next() a first time.  The decorator applies a first action somewhat like cocking a pistol, putting the generator or coroutine in the firing position, positioned at a first *yield*.

In [13]:
# -*- coding: utf-8 -*-
"""
Created on Thu Oct 13 13:48:52 2016

@author: Kirby Urner

David Beazley:
https://youtu.be/Z_OAlIhXziw?t=23m42s

Trial by division, but this time the primes coroutine acts 
more as a filter, passing qualified candidates through to
print_me, which writes to a file.
"""
import pprint

def coroutine(func):
    """
    Advances decorated generator function to the first yield
    """
    def start(*args, **kwargs):
        cr = func(*args, **kwargs)
        cr.send(None)  # or next(cr) or cr.__next__()
        return cr
    return start
        
@coroutine
def print_me(file_name):
    with open(file_name, 'w') as file_obj:
        while True:
            to_print = (yield)
            file_obj.write(str(to_print)+"\n")
    
@coroutine
def primes(target):
    _primes_so_far = [2]
    target.send(2)
    while True:
        candidate = (yield)
        for prev in _primes_so_far:
            if not divmod(candidate, prev)[1]:
                break
            if prev**2 > candidate:
                _primes_so_far.append(candidate)
                target.send(candidate)
                break

output = print_me("primes.txt")
p = primes(output)

for x in range(3, 200, 2):  # test odds 3-199
    p.send(x)

with open("primes.txt", 'r') as file_obj:
    print(", ".join(file_obj.read().split("\n"))[:-2])

2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199


In the top example, the generator provides a mechanism for advancing to a next odd number for testing, whereas the coroutine version consumes the odds we feed it, passing through those which pass the test for printing, as well as accumulating them internally.

### For Further Reading

Linked Jupyter Notebooks:

- [Public Key Cryptography](http://nbviewer.jupyter.org/github/4dsolutions/Python5/blob/master/Silicon%20Forest%20Math%20Series%20%7C%20RSA.ipynb) -- uses a generator
- [Playing With Cyphers](http://nbviewer.jupyter.org/github/4dsolutions/Python5/blob/master/Permutations.ipynb) -- Permutation type
- [Composition of Functions](http://nbviewer.jupyter.org/github/4dsolutions/Python5/blob/master/ComposingFunctions.ipynb) -- advanced Python
- [STEM Mathematics](http://nbviewer.jupyter.org/github/4dsolutions/Python5/blob/master/STEM%20Mathematics.ipynb)